# Week 2: word embeddings
How to get a sentiment from a sequence of numbers? It can be learned from a corpus of words (embedding).
The idea is that words and associate words are clustered as vectors in multidimensional space.

## The IMDB dataset
TensorFlow data services (TFDS) is a library that contains lots of data sets in lots of different categories.
We will use `imdb_reviews` dataset that contains 50k movie reviews categorized as positive or negative.

## Looking into the details

In [5]:
import tensorflow as tf
print(tf.__version__) # determine TF version, we will need eager execution which is enabled by default in TF 2.0
# tf.enable_eager_execution() # for TF 1.0

# Install the TFDS library:
# pip install -q tensorflow-datasets

import tensorflow_datasets as tfds
# get data and metadate from the imdb ds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

# Data is split in 25K samples for training and 25K samples for testing
import numpy as np
train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# Do some conversion to get arrays of sentences
for s,l in train_data: # iterate over train data to extract sentences and labels
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())

for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

# during the training we need a numpy array, so we convert them:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

# Tokenize the sentences
## first, define some hyperparams:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences) # fit tokenizer on a training set of data
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences) # replace strings containing the words with token values
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type) # pad/truncate the sentences until they are of the same length.

# Do the same for testing sequences; there should be more OOV token because we reuse the word index from training step
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length), # a key to performing text sentiment analysis in TF
    # the result of embedding is a 2d array with a len of the sentence and embedding dimension, that's why we have to flatten it:
    # tf.keras.layers.Flatten(),
    # or alternatively, use:
    tf.keras.layers.GlobalAveragePooling1D(), # averages across the vector to flatten it out; used often in NLP
    # Dense NN that performs classification:
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

num_epochs = 10
model.fit(
    padded,
    training_labels_final,
    epochs=num_epochs,
    validation_data=(testing_padded, testing_labels_final)
)

# Obtain and visualise the embeddings:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # (vocab_size, embedding_dim) -> (10000, 16) - 10K words inb corpus, working in a 16-dim array
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# Write vectors and metadata to files
import io
out_v = io.open('vects.csv', 'w', encoding='utf-8')
out_m = io.open('meta.csv', 'w', encoding='uft-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + '\n')
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

# Download files in Google Colab:
# try:
#     from google.colab import files
# except ImportError:
#     pass
# else:
#     files.download('vecs.tsv')
#     files.download('meta.tsv')

# projector.tensorflow.org -> Load data to visualize the results

2.3.1
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
782/782 [==============================] - 2s 2ms/step - loss: 0.5492 - accuracy: 0.7459 - val_loss: 0.3909 - val_accuracy: 0.8398